This notebook will generate the corpus that we'll use to train the new __DMC__.

In [1]:
import json
import os
import pickle
import sys

import numpy as np

module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config import PATH_BEST_MODELS  # noqa: E402
from src.config import PATH_INTERIM_CORPUS  # noqa: E402
from src.config import END_OF_POST_TOKEN, PICKLE_PROTOCOL  # noqa: E402
from src.models.model import EarlyModel, SimpleStopCriterion  # noqa: E402

In [2]:
INTERIM_DATASETS_PATH = os.path.join(PATH_INTERIM_CORPUS, "xml/depression")

In [3]:
CORPUS_PATH = os.path.join(INTERIM_DATASETS_PATH, "depression-train-raw.txt")
posts_list = []
with open(CORPUS_PATH) as f:
    for line in f:
        label, posts = line.split(maxsplit=1)
        if label == "positive":
            posts_list.append(posts.rstrip())

In [4]:
# List of tuples where the first element of each tuple is the index of the post in `posts_list` and the second index
# represents the manually label point in which the reading should stop.
# A value of -1 in the second index means that the users does not seem to be a positive user based on the posts only.
reviews = [
    (71, 3),
    (96, 1),
    (108, 23),
    (111, 9),
    (3, 12),
    (113, 36),
    (54, 1),
    (56, 8),
    (104, 1),
    (32, 1),
    (4, 1),
    (99, 117),
    (78, 207),
    (75, 518),
    (118, 196),
    (55, 89),
    (17, 6),
    (125, 21),
    (70, 1),
    (122, 240),
    (126, 1157),
    (59, -1),
    (16, -1),
    (46, -1),
    (74, -1),
    (24, 5),
    (129, 36),
    (94, 19),
    (97, 26),
    (36, 20),
    (107, 61),
    (58, 38),
    (26, -1),
    (47, 5),
    (69, 13),
    (61, 6),
    (21, -1),
    (28, 109),
    (7, -1),
    (133, 406),
    (132, 11),
    (72, 39),
    (41, 32),
    (20, 55),
    (11, 114),
    (80, 240),
    (76, 12),
    (90, -1),
    (35, 23),
    (77, 6),
]

In [5]:
# Remove users which don't seem positive.
reviews = [t for t in reviews if t[1] != -1]

-----------------------------------------------------

Generate a corpus to train the decision tree DMC model.

For this corpus, generate a sample for each post `i` from a user.
Each sample has all posts up to `i`.

Since for some representations, as the number of posts increases the information is "diluted", we cannot place the label `1` for all `posts > i` where `i` is the cutoff point. This can cause the model to learn things that are not appropriate.
But if we only consider positive those posts where we mark the cutoff point, we will obtain a very unbalanced corpus.
Therefore, after the cutoff point, we consider nine post more as positive.

In [6]:
NUMBER_POSTS_AFTER_STOP_POINT = 9
TRAIN_TEST_SPLIT = 0.5
TRAIN_SPLIT = int(len(reviews) * TRAIN_TEST_SPLIT)

In [7]:
raw_dmc_corpus_train = os.path.join(
    INTERIM_DATASETS_PATH, "depression-dmc-train-raw.txt"
)
if not os.path.exists(raw_dmc_corpus_train):
    for idx, stop_time in reviews[:TRAIN_SPLIT]:
        upper_limit = stop_time + NUMBER_POSTS_AFTER_STOP_POINT
        current_posts = posts_list[idx].split(END_OF_POST_TOKEN)[: upper_limit + 1]
        for i in range(1, min(upper_limit, len(current_posts)) + 1):
            label = "positive" if i >= stop_time else "negative"
            concatenated_post = END_OF_POST_TOKEN.join(current_posts[:i])
            with open(raw_dmc_corpus_train, "a", encoding="utf-8") as f:
                f.write(f"{label}\t{concatenated_post}\n")
else:
    print(f"The corpus {raw_dmc_corpus_train} was already created")

In [8]:
raw_dmc_corpus_test = os.path.join(INTERIM_DATASETS_PATH, "depression-dmc-test-raw.txt")
if not os.path.exists(raw_dmc_corpus_test):
    for idx, stop_time in reviews[TRAIN_SPLIT:]:
        upper_limit = stop_time + NUMBER_POSTS_AFTER_STOP_POINT
        current_posts = posts_list[idx].split(END_OF_POST_TOKEN)[: upper_limit + 1]
        for i in range(1, min(upper_limit, len(current_posts)) + 1):
            label = "positive" if i >= stop_time else "negative"
            concatenated_post = END_OF_POST_TOKEN.join(current_posts[:i])
            with open(raw_dmc_corpus_test, "a", encoding="utf-8") as f:
                f.write(f"{label}\t{concatenated_post}\n")
else:
    print(f"The corpus {raw_dmc_corpus_test} was already created")

-------

In [9]:
feature_names = [
    "current_probability",
    "avg_last_10_probabilities",
    "avg_last_5_probabilities",
    "median_last_10_probabilities",
    "current_delay",
    "num_words_information_gain_percentile_0_01",
    "num_words_chi2_percentile_0_015",
    "current_cpi_decision",
    "avg_last_10_cpi_decision",
]

dmc_corpus_feature_names = os.path.join(
    INTERIM_DATASETS_PATH, "depression-dmc-feature-names.json"
)
with open(dmc_corpus_feature_names, "w", encoding="utf-8") as fp:
    json.dump(fp=fp, obj=feature_names, indent="\t")

In [10]:
with open("depression_information_gain_words.json") as fp:
    depression_information_gain_words = json.load(fp=fp)

with open("depression_chi2_words.json") as fp:
    depression_chi2_words = json.load(fp=fp)

In [11]:
# Load base model to obtain the probabilities of the partial inputs.
# TODO: You need to update this.
model_path = os.path.join(
    PATH_BEST_MODELS, "positive_f1/reddit/depression/selected_models/03_model_SVC.json"
)

simple_criterion = SimpleStopCriterion(threshold=0.07, min_delay=1)

model = EarlyModel(
    path_to_model_information=model_path, stop_criterion=simple_criterion
)
model.clear_model_state()

In [12]:
y_train = []
x_train = []
groups_train = []
for idx, stop_time in reviews[:TRAIN_SPLIT]:
    last_probabilities = []
    last_decisions = []
    upper_limit = stop_time + NUMBER_POSTS_AFTER_STOP_POINT
    current_posts = posts_list[idx].split(END_OF_POST_TOKEN)[: upper_limit + 1]
    for i in range(1, min(upper_limit, len(current_posts)) + 1):
        label = 1 if i >= stop_time else 0
        y_train.append(label)
        raw_post = " ".join(current_posts[:i])
        concatenated_post = END_OF_POST_TOKEN.join(current_posts[:i])
        model.predict(documents_test=[concatenated_post], delay=i)
        predictions = model.predictions
        scores = model.probabilities
        if len(last_probabilities) < 10:
            last_probabilities.append(scores.item())
            last_decisions.append(predictions.item())
        else:
            last_probabilities = last_probabilities[1:] + [scores.item()]
            last_decisions = last_decisions[1:] + [predictions.item()]
        current_features = [
            scores.item(),  # CURRENT_PROBABILITY
            np.average(last_probabilities).item(),  # AVG_LAST_10_PROBABILITIES
            np.average(last_probabilities[-5:]).item(),  # AVG_LAST_5_PROBABILITIES
            np.median(last_probabilities).item(),  # MEDIAN_LAST_10_PROBABILITIES
            i,  # CURRENT_DELAY
            sum(
                1 if w in depression_information_gain_words else 0
                for w in raw_post.split()
            ),  # NUM_WORDS_INFORMATION_GAIN_PERCENTILE_0_01
            sum(
                1 if w in depression_chi2_words else 0 for w in raw_post.split()
            ),  # NUM_WORDS_CHI2_PERCENTILE_0_015
            predictions.item(),  # CURRENT_CPI_DECISION
            np.average(last_decisions).item(),  # AVG_LAST_10_CPI_DECISION
        ]
        x_train.append(current_features)
        groups_train.append(idx)
        model.clear_model_state()

2022-06-24 15:26:15.265374 | Current post number: 1 - Number of documents not finished: 1
2022-06-24 15:26:15.306360 | Current post number: 2 - Number of documents not finished: 1
2022-06-24 15:26:15.309033 | Current post number: 3 - Number of documents not finished: 1
2022-06-24 15:26:15.311599 | Current post number: 4 - Number of documents not finished: 1
2022-06-24 15:26:15.314788 | Current post number: 5 - Number of documents not finished: 1
2022-06-24 15:26:15.318079 | Current post number: 6 - Number of documents not finished: 1
2022-06-24 15:26:15.324414 | Current post number: 7 - Number of documents not finished: 1
2022-06-24 15:26:15.331402 | Current post number: 8 - Number of documents not finished: 1
2022-06-24 15:26:15.338503 | Current post number: 9 - Number of documents not finished: 1
2022-06-24 15:26:15.346112 | Current post number: 10 - Number of documents not finished: 1
2022-06-24 15:26:15.353987 | Current post number: 11 - Number of documents not finished: 1
2022-06-

2022-06-24 15:26:15.878148 | Current post number: 43 - Number of documents not finished: 1
2022-06-24 15:26:15.884729 | Current post number: 44 - Number of documents not finished: 1
2022-06-24 15:26:15.890959 | Current post number: 45 - Number of documents not finished: 1
2022-06-24 15:26:15.897199 | Current post number: 1 - Number of documents not finished: 1
2022-06-24 15:26:15.901693 | Current post number: 2 - Number of documents not finished: 1
2022-06-24 15:26:15.906508 | Current post number: 3 - Number of documents not finished: 1
2022-06-24 15:26:15.911643 | Current post number: 4 - Number of documents not finished: 1
2022-06-24 15:26:15.916782 | Current post number: 5 - Number of documents not finished: 1
2022-06-24 15:26:15.922015 | Current post number: 6 - Number of documents not finished: 1
2022-06-24 15:26:15.927245 | Current post number: 7 - Number of documents not finished: 1
2022-06-24 15:26:15.932507 | Current post number: 8 - Number of documents not finished: 1
2022-06

2022-06-24 15:26:16.288399 | Current post number: 39 - Number of documents not finished: 1
2022-06-24 15:26:16.294755 | Current post number: 40 - Number of documents not finished: 1
2022-06-24 15:26:16.300868 | Current post number: 41 - Number of documents not finished: 1
2022-06-24 15:26:16.307044 | Current post number: 42 - Number of documents not finished: 1
2022-06-24 15:26:16.313309 | Current post number: 43 - Number of documents not finished: 1
2022-06-24 15:26:16.319653 | Current post number: 44 - Number of documents not finished: 1
2022-06-24 15:26:16.326040 | Current post number: 45 - Number of documents not finished: 1
2022-06-24 15:26:16.332503 | Current post number: 46 - Number of documents not finished: 1
2022-06-24 15:26:16.339028 | Current post number: 47 - Number of documents not finished: 1
2022-06-24 15:26:16.345762 | Current post number: 48 - Number of documents not finished: 1
2022-06-24 15:26:16.352560 | Current post number: 49 - Number of documents not finished: 1

2022-06-24 15:26:17.124105 | Current post number: 12 - Number of documents not finished: 1
2022-06-24 15:26:17.129306 | Current post number: 13 - Number of documents not finished: 1
2022-06-24 15:26:17.134189 | Current post number: 14 - Number of documents not finished: 1
2022-06-24 15:26:17.139137 | Current post number: 15 - Number of documents not finished: 1
2022-06-24 15:26:17.144636 | Current post number: 16 - Number of documents not finished: 1
2022-06-24 15:26:17.150272 | Current post number: 17 - Number of documents not finished: 1
2022-06-24 15:26:17.156010 | Current post number: 18 - Number of documents not finished: 1
2022-06-24 15:26:17.161813 | Current post number: 19 - Number of documents not finished: 1
2022-06-24 15:26:17.167717 | Current post number: 20 - Number of documents not finished: 1
2022-06-24 15:26:17.173723 | Current post number: 21 - Number of documents not finished: 1
2022-06-24 15:26:17.179712 | Current post number: 22 - Number of documents not finished: 1

2022-06-24 15:26:17.961454 | Current post number: 118 - Number of documents not finished: 1
2022-06-24 15:26:17.971513 | Current post number: 119 - Number of documents not finished: 1
2022-06-24 15:26:17.981229 | Current post number: 120 - Number of documents not finished: 1
2022-06-24 15:26:17.991009 | Current post number: 121 - Number of documents not finished: 1
2022-06-24 15:26:18.000792 | Current post number: 122 - Number of documents not finished: 1
2022-06-24 15:26:18.010675 | Current post number: 123 - Number of documents not finished: 1
2022-06-24 15:26:18.020528 | Current post number: 124 - Number of documents not finished: 1
2022-06-24 15:26:18.030457 | Current post number: 125 - Number of documents not finished: 1
2022-06-24 15:26:18.040485 | Current post number: 126 - Number of documents not finished: 1
2022-06-24 15:26:18.050506 | Current post number: 127 - Number of documents not finished: 1
2022-06-24 15:26:18.060602 | Current post number: 128 - Number of documents not 

...

2022-06-24 15:26:59.676206 | Current post number: 760 - Number of documents not finished: 1
2022-06-24 15:26:59.726186 | Current post number: 761 - Number of documents not finished: 1
2022-06-24 15:26:59.775947 | Current post number: 762 - Number of documents not finished: 1
2022-06-24 15:26:59.825675 | Current post number: 763 - Number of documents not finished: 1
2022-06-24 15:26:59.875403 | Current post number: 764 - Number of documents not finished: 1
2022-06-24 15:26:59.926353 | Current post number: 765 - Number of documents not finished: 1
2022-06-24 15:26:59.976447 | Current post number: 766 - Number of documents not finished: 1
2022-06-24 15:27:00.026154 | Current post number: 767 - Number of documents not finished: 1
2022-06-24 15:27:00.075730 | Current post number: 768 - Number of documents not finished: 1
2022-06-24 15:27:00.125153 | Current post number: 769 - Number of documents not finished: 1
2022-06-24 15:27:00.175342 | Current post number: 770 - Number of documents not 

2022-06-24 15:27:04.580343 | Current post number: 854 - Number of documents not finished: 1
2022-06-24 15:27:04.634627 | Current post number: 855 - Number of documents not finished: 1
2022-06-24 15:27:04.688547 | Current post number: 856 - Number of documents not finished: 1
2022-06-24 15:27:04.742678 | Current post number: 857 - Number of documents not finished: 1
2022-06-24 15:27:04.797245 | Current post number: 858 - Number of documents not finished: 1
2022-06-24 15:27:04.852518 | Current post number: 859 - Number of documents not finished: 1
2022-06-24 15:27:04.906641 | Current post number: 860 - Number of documents not finished: 1
2022-06-24 15:27:04.960892 | Current post number: 861 - Number of documents not finished: 1
2022-06-24 15:27:05.015530 | Current post number: 862 - Number of documents not finished: 1
2022-06-24 15:27:05.072369 | Current post number: 863 - Number of documents not finished: 1
2022-06-24 15:27:05.127115 | Current post number: 864 - Number of documents not 

2022-06-24 15:27:09.810972 | Current post number: 946 - Number of documents not finished: 1
2022-06-24 15:27:09.870141 | Current post number: 947 - Number of documents not finished: 1
2022-06-24 15:27:09.928804 | Current post number: 948 - Number of documents not finished: 1
2022-06-24 15:27:09.987294 | Current post number: 949 - Number of documents not finished: 1
2022-06-24 15:27:10.046339 | Current post number: 950 - Number of documents not finished: 1
2022-06-24 15:27:10.105538 | Current post number: 951 - Number of documents not finished: 1
2022-06-24 15:27:10.164109 | Current post number: 952 - Number of documents not finished: 1
2022-06-24 15:27:10.223594 | Current post number: 953 - Number of documents not finished: 1
2022-06-24 15:27:10.283811 | Current post number: 954 - Number of documents not finished: 1
2022-06-24 15:27:10.343365 | Current post number: 955 - Number of documents not finished: 1
2022-06-24 15:27:10.402153 | Current post number: 956 - Number of documents not 

2022-06-24 15:27:15.410851 | Current post number: 1038 - Number of documents not finished: 1
2022-06-24 15:27:15.474382 | Current post number: 1039 - Number of documents not finished: 1
2022-06-24 15:27:15.536768 | Current post number: 1040 - Number of documents not finished: 1
2022-06-24 15:27:15.598902 | Current post number: 1041 - Number of documents not finished: 1
2022-06-24 15:27:15.661862 | Current post number: 1042 - Number of documents not finished: 1
2022-06-24 15:27:15.724963 | Current post number: 1043 - Number of documents not finished: 1
2022-06-24 15:27:15.787441 | Current post number: 1044 - Number of documents not finished: 1
2022-06-24 15:27:15.849938 | Current post number: 1045 - Number of documents not finished: 1
2022-06-24 15:27:15.913128 | Current post number: 1046 - Number of documents not finished: 1
2022-06-24 15:27:15.976414 | Current post number: 1047 - Number of documents not finished: 1
2022-06-24 15:27:16.039729 | Current post number: 1048 - Number of doc

2022-06-24 15:27:21.363671 | Current post number: 1130 - Number of documents not finished: 1
2022-06-24 15:27:21.432220 | Current post number: 1131 - Number of documents not finished: 1
2022-06-24 15:27:21.499919 | Current post number: 1132 - Number of documents not finished: 1
2022-06-24 15:27:21.567118 | Current post number: 1133 - Number of documents not finished: 1
2022-06-24 15:27:21.635248 | Current post number: 1134 - Number of documents not finished: 1
2022-06-24 15:27:21.702437 | Current post number: 1135 - Number of documents not finished: 1
2022-06-24 15:27:21.769522 | Current post number: 1136 - Number of documents not finished: 1
2022-06-24 15:27:21.836969 | Current post number: 1137 - Number of documents not finished: 1
2022-06-24 15:27:21.904841 | Current post number: 1138 - Number of documents not finished: 1
2022-06-24 15:27:21.973344 | Current post number: 1139 - Number of documents not finished: 1
2022-06-24 15:27:22.041240 | Current post number: 1140 - Number of doc

In [13]:
cant_pos_train = {}
for i, idx in enumerate(groups_train):
    if idx not in cant_pos_train:
        cant_pos_train[idx] = 0
    cant_pos_train[idx] += y_train[i]
cant_pos_train

{71: 10,
 96: 10,
 108: 10,
 111: 8,
 3: 10,
 113: 10,
 54: 10,
 56: 10,
 104: 10,
 32: 10,
 4: 10,
 99: 10,
 78: 10,
 75: 1,
 118: 10,
 55: 10,
 17: 10,
 125: 10,
 70: 10,
 122: 2,
 126: 10}

In [14]:
x_train = np.array(x_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)

In [15]:
dmc_corpus_train = os.path.join(INTERIM_DATASETS_PATH, "depression-dmc-train.pkl")
with open(dmc_corpus_train, "wb") as fp:
    pickle.dump((x_train, y_train, groups_train), fp, protocol=PICKLE_PROTOCOL)

------

In [16]:
y_test = []
x_test = []
groups_test = []
for idx, stop_time in reviews[TRAIN_SPLIT:]:
    last_probabilities = []
    last_decisions = []
    upper_limit = stop_time + NUMBER_POSTS_AFTER_STOP_POINT
    current_posts = posts_list[idx].split(END_OF_POST_TOKEN)[: upper_limit + 1]
    for i in range(1, min(upper_limit, len(current_posts)) + 1):
        label = 1 if i >= stop_time else 0
        y_test.append(label)
        raw_post = " ".join(current_posts[:i])
        concatenated_post = END_OF_POST_TOKEN.join(current_posts[:i])
        model.predict(documents_test=[concatenated_post], delay=i)
        predictions = model.predictions
        scores = model.probabilities
        if len(last_probabilities) < 10:
            last_probabilities.append(scores.item())
            last_decisions.append(predictions.item())
        else:
            last_probabilities = last_probabilities[1:] + [scores.item()]
            last_decisions = last_decisions[1:] + [predictions.item()]
        current_features = [
            scores.item(),  # CURRENT_PROBABILITY
            np.average(last_probabilities).item(),  # AVG_LAST_10_PROBABILITIES
            np.average(last_probabilities[-5:]).item(),  # AVG_LAST_5_PROBABILITIES
            np.median(last_probabilities).item(),  # MEDIAN_LAST_10_PROBABILITIES
            i,  # CURRENT_DELAY
            sum(
                1 if w in depression_information_gain_words else 0
                for w in raw_post.split()
            ),  # NUM_WORDS_INFORMATION_GAIN_PERCENTILE_0_01
            sum(
                1 if w in depression_chi2_words else 0 for w in raw_post.split()
            ),  # NUM_WORDS_CHI2_PERCENTILE_0_015
            predictions.item(),  # CURRENT_CPI_DECISION
            np.average(last_decisions).item(),  # AVG_LAST_10_CPI_DECISION
        ]
        x_test.append(current_features)
        groups_test.append(idx)
        model.clear_model_state()

2022-06-24 15:27:24.615487 | Current post number: 1 - Number of documents not finished: 1
2022-06-24 15:27:24.620159 | Current post number: 2 - Number of documents not finished: 1
2022-06-24 15:27:24.623623 | Current post number: 3 - Number of documents not finished: 1
2022-06-24 15:27:24.627311 | Current post number: 4 - Number of documents not finished: 1
2022-06-24 15:27:24.631181 | Current post number: 5 - Number of documents not finished: 1
2022-06-24 15:27:24.635642 | Current post number: 6 - Number of documents not finished: 1
2022-06-24 15:27:24.640287 | Current post number: 7 - Number of documents not finished: 1
2022-06-24 15:27:24.645286 | Current post number: 8 - Number of documents not finished: 1
2022-06-24 15:27:24.650690 | Current post number: 9 - Number of documents not finished: 1
2022-06-24 15:27:24.655831 | Current post number: 10 - Number of documents not finished: 1
2022-06-24 15:27:24.660890 | Current post number: 11 - Number of documents not finished: 1
2022-06-

2022-06-24 15:27:25.228369 | Current post number: 14 - Number of documents not finished: 1
2022-06-24 15:27:25.233427 | Current post number: 15 - Number of documents not finished: 1
2022-06-24 15:27:25.238259 | Current post number: 16 - Number of documents not finished: 1
2022-06-24 15:27:25.243162 | Current post number: 17 - Number of documents not finished: 1
2022-06-24 15:27:25.248168 | Current post number: 18 - Number of documents not finished: 1
2022-06-24 15:27:25.253218 | Current post number: 19 - Number of documents not finished: 1
2022-06-24 15:27:25.258361 | Current post number: 20 - Number of documents not finished: 1
2022-06-24 15:27:25.265102 | Current post number: 21 - Number of documents not finished: 1
2022-06-24 15:27:25.272013 | Current post number: 22 - Number of documents not finished: 1
2022-06-24 15:27:25.279396 | Current post number: 23 - Number of documents not finished: 1
2022-06-24 15:27:25.286821 | Current post number: 24 - Number of documents not finished: 1

2022-06-24 15:27:25.840435 | Current post number: 32 - Number of documents not finished: 1
2022-06-24 15:27:25.844442 | Current post number: 33 - Number of documents not finished: 1
2022-06-24 15:27:25.848188 | Current post number: 34 - Number of documents not finished: 1
2022-06-24 15:27:25.852002 | Current post number: 35 - Number of documents not finished: 1
2022-06-24 15:27:25.855807 | Current post number: 36 - Number of documents not finished: 1
2022-06-24 15:27:25.859970 | Current post number: 37 - Number of documents not finished: 1
2022-06-24 15:27:25.864286 | Current post number: 38 - Number of documents not finished: 1
2022-06-24 15:27:25.868881 | Current post number: 39 - Number of documents not finished: 1
2022-06-24 15:27:25.873729 | Current post number: 40 - Number of documents not finished: 1
2022-06-24 15:27:25.878708 | Current post number: 41 - Number of documents not finished: 1
2022-06-24 15:27:25.883763 | Current post number: 42 - Number of documents not finished: 1

2022-06-24 15:27:26.252473 | Current post number: 31 - Number of documents not finished: 1
2022-06-24 15:27:26.256431 | Current post number: 32 - Number of documents not finished: 1
2022-06-24 15:27:26.260030 | Current post number: 33 - Number of documents not finished: 1
2022-06-24 15:27:26.263647 | Current post number: 34 - Number of documents not finished: 1
2022-06-24 15:27:26.267239 | Current post number: 35 - Number of documents not finished: 1
2022-06-24 15:27:26.270834 | Current post number: 36 - Number of documents not finished: 1
2022-06-24 15:27:26.274597 | Current post number: 37 - Number of documents not finished: 1
2022-06-24 15:27:26.278294 | Current post number: 38 - Number of documents not finished: 1
2022-06-24 15:27:26.282012 | Current post number: 39 - Number of documents not finished: 1
2022-06-24 15:27:26.285748 | Current post number: 40 - Number of documents not finished: 1
2022-06-24 15:27:26.289659 | Current post number: 41 - Number of documents not finished: 1

2022-06-24 15:27:26.870555 | Current post number: 34 - Number of documents not finished: 1
2022-06-24 15:27:26.878907 | Current post number: 35 - Number of documents not finished: 1
2022-06-24 15:27:26.886842 | Current post number: 36 - Number of documents not finished: 1
2022-06-24 15:27:26.894820 | Current post number: 37 - Number of documents not finished: 1
2022-06-24 15:27:26.902844 | Current post number: 38 - Number of documents not finished: 1
2022-06-24 15:27:26.911171 | Current post number: 39 - Number of documents not finished: 1
2022-06-24 15:27:26.919468 | Current post number: 40 - Number of documents not finished: 1
2022-06-24 15:27:26.927757 | Current post number: 41 - Number of documents not finished: 1
2022-06-24 15:27:26.935903 | Current post number: 42 - Number of documents not finished: 1
2022-06-24 15:27:26.944796 | Current post number: 43 - Number of documents not finished: 1
2022-06-24 15:27:26.953605 | Current post number: 44 - Number of documents not finished: 1

...

2022-06-24 15:27:33.560362 | Current post number: 31 - Number of documents not finished: 1
2022-06-24 15:27:33.565752 | Current post number: 32 - Number of documents not finished: 1
2022-06-24 15:27:33.570802 | Current post number: 33 - Number of documents not finished: 1
2022-06-24 15:27:33.575849 | Current post number: 34 - Number of documents not finished: 1
2022-06-24 15:27:33.580883 | Current post number: 35 - Number of documents not finished: 1
2022-06-24 15:27:33.585956 | Current post number: 36 - Number of documents not finished: 1
2022-06-24 15:27:33.591123 | Current post number: 37 - Number of documents not finished: 1
2022-06-24 15:27:33.596328 | Current post number: 38 - Number of documents not finished: 1
2022-06-24 15:27:33.601591 | Current post number: 39 - Number of documents not finished: 1
2022-06-24 15:27:33.607253 | Current post number: 40 - Number of documents not finished: 1
2022-06-24 15:27:33.612922 | Current post number: 41 - Number of documents not finished: 1

2022-06-24 15:27:34.182460 | Current post number: 64 - Number of documents not finished: 1
2022-06-24 15:27:34.192319 | Current post number: 65 - Number of documents not finished: 1
2022-06-24 15:27:34.201913 | Current post number: 66 - Number of documents not finished: 1
2022-06-24 15:27:34.211560 | Current post number: 67 - Number of documents not finished: 1
2022-06-24 15:27:34.221383 | Current post number: 68 - Number of documents not finished: 1
2022-06-24 15:27:34.231389 | Current post number: 69 - Number of documents not finished: 1
2022-06-24 15:27:34.241327 | Current post number: 70 - Number of documents not finished: 1
2022-06-24 15:27:34.251379 | Current post number: 71 - Number of documents not finished: 1
2022-06-24 15:27:34.261389 | Current post number: 72 - Number of documents not finished: 1
2022-06-24 15:27:34.271426 | Current post number: 73 - Number of documents not finished: 1
2022-06-24 15:27:34.281512 | Current post number: 74 - Number of documents not finished: 1

2022-06-24 15:27:35.244957 | Current post number: 52 - Number of documents not finished: 1
2022-06-24 15:27:35.254252 | Current post number: 53 - Number of documents not finished: 1
2022-06-24 15:27:35.263526 | Current post number: 54 - Number of documents not finished: 1
2022-06-24 15:27:35.272783 | Current post number: 55 - Number of documents not finished: 1
2022-06-24 15:27:35.282100 | Current post number: 56 - Number of documents not finished: 1
2022-06-24 15:27:35.291416 | Current post number: 57 - Number of documents not finished: 1
2022-06-24 15:27:35.300770 | Current post number: 58 - Number of documents not finished: 1
2022-06-24 15:27:35.310176 | Current post number: 59 - Number of documents not finished: 1
2022-06-24 15:27:35.319751 | Current post number: 60 - Number of documents not finished: 1
2022-06-24 15:27:35.329520 | Current post number: 61 - Number of documents not finished: 1
2022-06-24 15:27:35.339255 | Current post number: 62 - Number of documents not finished: 1

2022-06-24 15:27:36.516709 | Current post number: 154 - Number of documents not finished: 1
2022-06-24 15:27:36.532498 | Current post number: 155 - Number of documents not finished: 1
2022-06-24 15:27:36.548015 | Current post number: 156 - Number of documents not finished: 1
2022-06-24 15:27:36.563494 | Current post number: 157 - Number of documents not finished: 1
2022-06-24 15:27:36.579073 | Current post number: 158 - Number of documents not finished: 1
2022-06-24 15:27:36.594772 | Current post number: 159 - Number of documents not finished: 1
2022-06-24 15:27:36.610722 | Current post number: 160 - Number of documents not finished: 1
2022-06-24 15:27:36.626731 | Current post number: 161 - Number of documents not finished: 1
2022-06-24 15:27:36.642977 | Current post number: 162 - Number of documents not finished: 1
2022-06-24 15:27:36.658838 | Current post number: 163 - Number of documents not finished: 1
2022-06-24 15:27:36.674815 | Current post number: 164 - Number of documents not 

2022-06-24 15:27:38.266375 | Current post number: 2 - Number of documents not finished: 1
2022-06-24 15:27:38.269080 | Current post number: 3 - Number of documents not finished: 1
2022-06-24 15:27:38.271502 | Current post number: 4 - Number of documents not finished: 1
2022-06-24 15:27:38.273912 | Current post number: 5 - Number of documents not finished: 1
2022-06-24 15:27:38.276361 | Current post number: 6 - Number of documents not finished: 1
2022-06-24 15:27:38.279239 | Current post number: 7 - Number of documents not finished: 1
2022-06-24 15:27:38.282120 | Current post number: 8 - Number of documents not finished: 1
2022-06-24 15:27:38.285684 | Current post number: 9 - Number of documents not finished: 1
2022-06-24 15:27:38.289307 | Current post number: 10 - Number of documents not finished: 1
2022-06-24 15:27:38.292991 | Current post number: 11 - Number of documents not finished: 1
2022-06-24 15:27:38.296729 | Current post number: 12 - Number of documents not finished: 1
2022-06

In [17]:
cant_pos_test = {}
for i, idx in enumerate(groups_test):
    if idx not in cant_pos_test:
        cant_pos_test[idx] = 0
    cant_pos_test[idx] += y_test[i]
cant_pos_test

{24: 10,
 129: 10,
 94: 5,
 97: 10,
 36: 10,
 107: 10,
 58: 10,
 47: 10,
 69: 10,
 61: 10,
 28: 10,
 133: 10,
 132: 2,
 72: 10,
 41: 10,
 20: 10,
 11: 10,
 80: 10,
 76: 10,
 35: 10,
 77: 10}

In [18]:
x_test = np.array(x_test)
y_test = np.array(y_test)
groups_test = np.array(groups_test)

In [19]:
dmc_corpus_test = os.path.join(INTERIM_DATASETS_PATH, "depression-dmc-test.pkl")
with open(dmc_corpus_test, "wb") as fp:
    pickle.dump((x_test, y_test, groups_test), fp, protocol=PICKLE_PROTOCOL)